In [1]:
import torch
import random
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from sklearn.model_selection import train_test_split

from utils.util import get_image_list, dataset_list, PathDataset, PathDataset_v2, make_task

from avalanche.benchmarks.utils import AvalancheTensorDataset
from avalanche.training.supervised import AGEM
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.benchmarks.generators import nc_benchmark, paths_benchmark
from avalanche.benchmarks.utils import AvalancheDataset, AvalancheDatasetType
from avalanche.benchmarks.utils.dataset_definitions import IDatasetWithTargets
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy
from avalanche.benchmarks.generators import (
    filelist_scenario, dataset_scenario, tensor_scenario, paths_scenario
)

/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False
cudnn.deterministic = True

In [3]:
transform = transforms.Compose([
    # transforms.Resize(224),
    transforms.ToTensor()
    ])

In [4]:
incremental = 500
data_path = "../Data/CASIA-WebFace/" 

dataset = get_image_list(data_path)

image_dataset = []
label_dataset = []
incremental_list = list(range(incremental))

for idx, label in enumerate(dataset[1]):
    if label in incremental_list:
        image_dataset.append(dataset[0][idx])
        label_dataset.append(dataset[1][idx])

# x_train, x_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.2, shuffle=True, random_state=seed, stratify=dataset[1])
x_train, x_test, y_train, y_test = train_test_split(image_dataset, label_dataset, test_size=0.2, shuffle=True, random_state=seed, stratify=label_dataset)

train_set = dataset_list(x_train, y_train)
test_set = dataset_list(x_test, y_test)

print("train : ", len(y_train))
print("test : ", len(y_test))

train :  39374
test :  9844


In [5]:
transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

In [6]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# num_class = 10575
# train_task = make_task(incremental, y_train)
# test_task = make_task(incremental, y_test)

# train_set = PathDataset(train_set, transforms_group, "train", train_task)
# test_set = PathDataset(test_set, transforms_group, "eval", test_task)
train_set = PathDataset_v2(train_set, transforms_group, "train")
test_set = PathDataset_v2(test_set, transforms_group, "eval")

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=True
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=incremental)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-1)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs_agem_casia")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceTime(),
    loggers=[interactive_logger, tensor_logger])

2022-08-09 15:38:01.089863: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


## A-GEM

In [8]:
train_batch = 10
eval_batch = 10
epoch = 1
memory_size = 256

strategies = AGEM(model, optimizer, criterion, patterns_per_exp=memory_size, sample_size=memory_size, train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [10]:
print("Starting experiment...")
results = []
for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))

Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████| 3/3 [00:00<00:00, 15.50it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6296
100%|██████████| 3/3 [00:00<00:00, 19.87it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 3/3 [00:00<00:00, 20.45it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 3/3 [00:00<00:00, 20.82it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 3/3 [00:00<00:00, 20.56it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 3/3 [00:00<00:00, 20.39it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 3/3 [00:00<00:00, 18.59it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 3/3 [00:00<00:00, 20.93it/s]
Epoch 7 ended.
	Top1_Acc_Epoch